In [1]:
import torch
import gpytorch
import time

%pdb

Automatic pdb calling has been turned ON


In [2]:
n = 1000

train_x = torch.randn(n, 1)
kernel = gpytorch.kernels.keops.SpectralGPKernel(train_x, register_latent_params=True)
rhs = torch.randn(n, 1)

In [3]:
list(kernel.named_parameters()) # latent params requires gradients?

[('latent_params', Parameter containing:
  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.], requires_grad=True))]

In [4]:
start = time.time()
res_simple = kernel(train_x).add_jitter(1e-3).inv_quad_logdet(rhs,logdet=True)
end = time.time()
print('elapsed time: ', end - start)

elapsed time:  0.18179726600646973


In [5]:
start = time.time()
res_simple = kernel(train_x).add_jitter(1e-3).inv_quad_logdet(rhs,logdet=True)
end = time.time()
print('elapsed time: ', end - start)

elapsed time:  0.11197876930236816


In [6]:
start = time.time()
dist = gpytorch.distributions.MultivariateNormal(torch.zeros_like(train_x).view(-1), 
                                                 kernel(train_x).add_jitter(1e-3))
logprob = dist.log_prob(rhs.view(-1))
end = time.time()
print('elapsed time:', end - start)

elapsed time: 0.1026155948638916


In [7]:
start = time.time()
dist = gpytorch.distributions.MultivariateNormal(torch.zeros_like(train_x).view(-1), 
                                                 kernel(train_x).add_jitter(1e-3))
logprob = dist.log_prob(rhs.view(-1))
end = time.time()
print('elapsed time:', end - start)

elapsed time: 0.09623980522155762


In [8]:
logprob # note that theres no gradients

tensor(-450126.4688)

In [9]:
class SpectralGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(SpectralGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.keops.SpectralGPKernel(train_x, register_latent_params=True)
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [10]:
model = SpectralGPModel(train_x, rhs.view(-1), gpytorch.likelihoods.GaussianLikelihood())

In [11]:
tmp = model(train_x).log_prob(rhs.view(-1))

/home/wesley/Documents/Code/wjm_gpytorch/gpytorch/utils/linear_cg.py:315: UserWarning: CG terminated in 1000 iterations with average residual norm 9805591.0 which is larger than the tolerance of 1 specified by gpytorch.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a gpytorch.settings.max_cg_iterations(value) context.
  " a gpytorch.settings.max_cg_iterations(value) context.".format(k + 1, residual_norm.mean(), tolerance)


In [12]:
tmp.backward()

In [13]:
model.covar_module.latent_params.grad # the gradients arent flowing here either